## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,71.65,78,20,3.44,few clouds
1,1,Tasiilaq,GL,65.6145,-37.6368,39.36,96,100,14.83,light rain
2,2,Sitka,US,57.0531,-135.3300,48.09,76,90,11.50,light rain
3,3,Qaanaaq,GL,77.4840,-69.3632,32.50,93,38,6.04,scattered clouds
4,4,Atuona,PF,-9.8000,-139.0333,78.51,71,13,10.94,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Avarua,CK,71.65,-21.2078,-159.7750,few clouds,
4,Atuona,PF,78.51,-9.8000,-139.0333,few clouds,
8,Rikitea,PF,76.89,-23.1203,-134.9692,broken clouds,
14,Cockburn Town,TC,79.41,21.4612,-71.1419,few clouds,
19,Celestun,MX,87.15,20.8667,-90.4000,clear sky,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        pass
        

In [9]:
hotel_df.loc[hotel_df["Hotel Name"] == ""]

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
47,Yumen,CN,71.01,40.2833,97.2000,few clouds,
54,Mtambile,TZ,77.23,-5.3833,39.7000,scattered clouds,
70,Kamenka,RU,72.73,51.3223,42.7678,few clouds,
141,Businga,CD,87.37,3.3333,20.8833,overcast clouds,
182,Merauke,ID,75.00,-8.4667,140.3333,broken clouds,
304,Avera,US,80.47,33.1940,-82.5271,overcast clouds,
314,Pangoa,PE,83.79,-12.1167,-73.0000,few clouds,
358,Kaniama,CD,78.04,-7.5667,24.1833,scattered clouds,
374,Ouadda,CF,85.53,8.0777,22.4007,scattered clouds,
480,Dombarovskiy,RU,76.68,50.7587,59.5386,broken clouds,


In [10]:
# 7 Drop any row where Hotel Name could not be found
clean_hotel_df = hotel_df.drop(hotel_df.loc[hotel_df["Hotel Name"] == ""].index)

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,Avarua,CK,71.65,-21.2078,-159.7750,few clouds,Paradise Inn
4,Atuona,PF,78.51,-9.8000,-139.0333,few clouds,Villa Enata
8,Rikitea,PF,76.89,-23.1203,-134.9692,broken clouds,Pension Maro'i
14,Cockburn Town,TC,79.41,21.4612,-71.1419,few clouds,Osprey Beach Hotel
19,Celestun,MX,87.15,20.8667,-90.4000,clear sky,Posada Lilia
...,...,...,...,...,...,...,...
652,La Rioja,ES,76.78,42.2500,-2.5000,light rain,Casa Concejos
653,Bojnurd,IR,89.55,37.4747,57.3290,few clouds,Homag Commercial & Social Complex
655,Hvide Sande,DK,70.38,56.0045,8.1294,few clouds,Hvide Sande Hotel
656,Carutapera,BR,86.23,-1.1950,-46.0200,overcast clouds,Lidera


In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))